# Frequency Analysis on Caesar Cipher

## Classical Ciphers

https://en.wikipedia.org/wiki/Classical_cipher

## Encryption and Decryption

In the Caesar Cipher, the encryption method is substituting each letter of the plain text with another letter using a numeric integer key. I will use the letter `k` to denote the key and the letter `p` to denote the plain text. The substitution is performed in the following manner: each letter in `p` is replaced with a letter that is `k` letters after it in the alphabet, for example with `k=1` the letter `A` would be substituted with the letter `B`, the letter `E` would be substituted with the letter `F` and the letter `Z` would be substituted with the letter `A`. From this explanation we can derive a mathematical function for **decryption** and **encryption** of the Caesar Cipher.

For `x` is the letter to be encrypted, we can use the formula `E(x) = (x + k) mod 26` to encrypt and `E(x) = (x - k) mod 26` to decrypt.

Note: `k≥26` is equivalent to `k = k mod 26` because the size of the English alphabet is 26.

Ref: https://medium.com/@Nougat-Waffle/caesar-cipher-and-frequency-analysis-with-python-635b04e0186f


In [ ]:
from string import ascii_lowercase

ALPHABET = ascii_lowercase  # abcdefghijklmnopqrstuvwxyz
ALPHABET_SIZE = len(ALPHABET)
print(f"Alphabet: {ALPHABET}, Size: {ALPHABET_SIZE}")


def _crypt(text: str, key: int) -> str:
    """
    Encrypt or decrypt the text. Pass a negative key to decrypt.
    """
    output = ""

    for char in text:
        # If the character is not in the english alphabet don't change it.
        if not char.isalpha():
            output += char
            continue

        index = ALPHABET.index(char.lower())
        # E(x) = (x +/- k) mod 26
        new_char = ALPHABET[(index + key) % ALPHABET_SIZE]

        # Setting the right case for the letter and adding it to the output
        output += new_char.upper() if char.isupper() else new_char

    return output


def encrypt(plain_text: str, key: int) -> str:
    return _crypt(plain_text, key)


def decrypt(cipher_text: str, key: int) -> str:
    # Ensure that the key is negative.
    key = -abs(key)
    return _crypt(cipher_text, key)

In [ ]:
encrypt("a", 1)

In [ ]:
decrypt("a", 9)

## Frequency Analysis

> In cryptanalysis, frequency analysis is the study of the frequency of letters or groups of letters in a ciphertext. The method is used as an aid to breaking classical ciphers.

In [ ]:
from urllib.request import urlopen

# from collections import Counter
import matplotlib.pyplot as plt

# def calculate_letter_frequency(text: str) -> dict:
#     letter_freq: dict = {}
#     text = text.lower()
#     total_count = 0
#     for char in text:
#         if char in ascii_lowercase and char not in letter_freq:
#             letter_freq[char] = text.count(char)
#     return letter_freq


def calculate_letter_frequency(text: str) -> dict:
    letter_freq: dict = {}
    text = text.lower()
    total_count = 0
    for char in text:
        if char in ascii_lowercase:
            total_count += 1
            if char not in letter_freq:
                letter_freq[char] = text.count(char)
    for l, f in letter_freq.items():
        letter_freq[l] = f * 100 / total_count
    return letter_freq


def plot_histogram(dictionary, title="Frequency analysis"):
    sorted_items = sorted(dictionary.items(), key=lambda item: item[1], reverse=True)
    keys = [item[0] for item in sorted_items]
    values = [item[1] for item in sorted_items]

    plt.bar(keys, values)
    plt.xlabel("Characters")
    plt.ylabel("Frequency %")
    plt.title(title)
    plt.show()


url_hamlet = "https://www.gutenberg.org/cache/epub/1787/pg1787.txt"
with urlopen(url_hamlet) as f:
    text = f.read().decode("utf-8").strip()
    # print(Counter(text))
    letter_freq = calculate_letter_frequency(text)
    print(letter_freq)
    plot_histogram(letter_freq)

url_romeo_and_juliet = "https://www.gutenberg.org/cache/epub/47960/pg47960.txt"
with urlopen(url_romeo_and_juliet) as f:
    text = f.read().decode("utf-8").strip()
    letter_freq = calculate_letter_frequency(text)
    print(letter_freq)
    plot_histogram(letter_freq)

In [ ]:
# Taken from https://en.wikipedia.org/wiki/Letter_frequency.
# Values are percentage.
LETTER_FREQUENCY = {
    "e": 12.7,
    "t": 9.1,
    "a": 8.2,
    "o": 7.5,
    "i": 7.0,
    "n": 6.7,
    "s": 6.3,
    "h": 6.1,
    "r": 6.0,
    "d": 4.25,
    "l": 4.0,
    "c": 2.8,
    "u": 2.8,
    "m": 2.4,
    "w": 2.4,
    "f": 2.2,
    "g": 2.0,
    "y": 2.0,
    "p": 1.9,
    "b": 1.5,
    "v": 0.98,
    "k": 0.77,
    "j": 0.15,
    "x": 0.15,
    "q": 0.095,
    "z": 0.074,
}
# print(sum(LETTER_FREQUENCY.values()))
plot_histogram(LETTER_FREQUENCY)

TODO: add more documentation


In [ ]:
from math import inf


def calculate_difference(text: str) -> float:
    letter_freq = calculate_letter_frequency(text)
    difference = (
        sum([abs(letter_freq.get(letter, 0) - LETTER_FREQUENCY[letter]) for letter in ALPHABET]) / ALPHABET_SIZE
    )
    return difference


def break_cipher(cipher_text: str) -> int:
    lowest_difference = inf
    encryption_key = 0

    for key in range(1, ALPHABET_SIZE):
        current_plain_text = decrypt(cipher_text, key)
        current_difference = calculate_difference(current_plain_text)
        if current_difference < lowest_difference:
            lowest_difference = current_difference
            encryption_key = key
            print(f"break_cipher -> encryption_key: {encryption_key}, lowest_difference: {lowest_difference}")
    return encryption_key

## Demo

In [ ]:
# Ex: Brvruja rmnjb jan dbnm rw vxmnaw mjcj-lxvyanbbrxw cnlqwrzdnb bdlq jb Qdoovjw lxmrwp.
# https://cryptii.com/pipes/caesar-cipher
print("")
example_encrypted_text = input("Give me a cipher text to break:")
print("")

letter_freq = calculate_letter_frequency(example_encrypted_text)
plot_histogram(letter_freq, title="Frequency analysis of given encrypted text")
plot_histogram(LETTER_FREQUENCY, title="Frequency analysis of English alphabet")

encryption_key = break_cipher(example_encrypted_text)
print("")
decrypted_text = decrypt(example_encrypted_text, encryption_key)
print(f"Decrypted text: {decrypted_text}")

In [ ]:
# example = "My name is John."
# print(example)
# letter_freq = calculate_letter_frequency(example)
# plot_histogram(letter_freq)

# encrypted_text = encrypt(example, 9)
# print(encrypted_text)
# letter_freq = calculate_letter_frequency(encrypted_text)
# plot_histogram(letter_freq)

# encryption_key = break_cipher(encrypted_text)
# print(encryption_key)

# decrypt(encrypted_text, encryption_key)

In [ ]:
# # https://cryptii.com/pipes/caesar-cipher
# # If he had anything confidential to say, he wrote it in cipher, that is, by so changing the order of the letters of the alphabet, that not a word could be made out.
# example = "Vs ur unq nalguvat pbasvqragvny gb fnl, ur jebgr vg va pvcure, gung vf, ol fb punatvat gur beqre bs gur yrggref bs gur nycunorg, gung abg n jbeq pbhyq or znqr bhg."
# encryption_key = break_cipher(example)
# print(encryption_key)
# decrypt(example, encryption_key)